# Safe Integration of Generated OpenAPI Tools

## 🎯 Objective
Safely integrate auto-generated tool configurations from OpenAPI specs into GenAI Toolbox using **multi-file configuration strategy**.

## 📊 Key Discovery
**GenAI Toolbox supports loading multiple YAML files from a folder!**

This notebook will help you:
1. ✅ Use folder-based configuration (best practice)
2. ✅ Integrate Echo API first (safe approach)
3. ✅ Verify everything works before adding more
4. ✅ Scale to Statistics and Public Register APIs

## 📚 Reference Documents
- `QUICK_ANSWER.md` - Summary
- `MULTI_FILE_STRATEGY.md` - Complete strategy
- `ECHO_MIGRATION_GUIDE.md` - Step-by-step guide

## 1️⃣ Setup & Imports

In [ ]:
import yaml
import shutil
from pathlib import Path
from datetime import datetime
import subprocess
import time

# Paths
WORKSPACE = Path(r"C:\Users\rjjaf\_Projects\orkhon")
CONFIG_DIR = WORKSPACE / "backend" / "toolbox" / "config"
GENERATED_DIR = WORKSPACE / "backend" / "apis" / "dnb" / "generated"
DOCKER_COMPOSE_FILE = WORKSPACE / "backend" / "toolbox" / "docker-compose.dev.yml"

print("✅ Imports loaded")
print(f"📂 Config directory: {CONFIG_DIR}")
print(f"📂 Generated directory: {GENERATED_DIR}")
print(f"📄 Docker compose: {DOCKER_COMPOSE_FILE}")

## 2️⃣ Verify Generated Files Exist

In [ ]:
# Check generated files
generated_files = {
    "echo": GENERATED_DIR / "tools.echo.generated.yaml",
    "statistics": GENERATED_DIR / "tools.statistics.generated.yaml",
    "public-register": GENERATED_DIR / "tools.public-register.generated.yaml"
}

print("🔍 Checking generated files...")
for api_name, file_path in generated_files.items():
    if file_path.exists():
        print(f"  ✅ {api_name}: {file_path.name}")
        
        # Load and show summary
        with open(file_path) as f:
            config = yaml.safe_load(f)
        num_sources = len(config.get('sources', []))
        num_tools = len(config.get('tools', []))
        print(f"      Sources: {num_sources}, Tools: {num_tools}")
    else:
        print(f"  ❌ {api_name}: NOT FOUND - {file_path}")

## 3️⃣ Backup Existing Configuration

**SAFETY FIRST!** Always backup before making changes.

In [ ]:
# Create backup with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
original_file = CONFIG_DIR / "tools.dev.yaml"
backup_file = CONFIG_DIR / f"tools.dev.yaml.backup_{timestamp}"

if original_file.exists():
    shutil.copy2(original_file, backup_file)
    print(f"✅ Backup created: {backup_file.name}")
    
    # Show file size
    size_kb = backup_file.stat().st_size / 1024
    print(f"   Size: {size_kb:.2f} KB")
else:
    print(f"⚠️  Original file not found: {original_file}")

## 4️⃣ Copy Echo API (Start Safe)

**Strategy**: Start with just Echo API to verify multi-file config works.

In [ ]:
# Copy Echo API generated file to config directory
echo_generated = GENERATED_DIR / "tools.echo.generated.yaml"
echo_config = CONFIG_DIR / "dnb-echo.yaml"

if echo_generated.exists():
    shutil.copy2(echo_generated, echo_config)
    print(f"✅ Copied: {echo_generated.name}")
    print(f"   → {echo_config}")
    
    # Show content summary
    with open(echo_config) as f:
        config = yaml.safe_load(f)
    
    print(f"\n📊 Echo API Configuration:")
    print(f"   Sources: {len(config.get('sources', []))}")
    print(f"   Tools: {len(config.get('tools', []))}")
    
    # Show IDs
    for source in config.get('sources', []):
        print(f"     - Source ID: {source.get('id')}")
    for tool in config.get('tools', []):
        print(f"     - Tool ID: {tool.get('id')}")
else:
    print(f"❌ Generated file not found: {echo_generated}")

## 5️⃣ Check for ID Conflicts

Before deploying, validate there are no duplicate IDs across all YAML files.

In [ ]:
def check_conflicts(config_dir):
    """Check for duplicate source/tool IDs across all YAML files"""
    all_sources = {}
    all_tools = {}
    
    yaml_files = list(config_dir.glob("*.yaml")) + list(config_dir.glob("*.yml"))
    
    print(f"🔍 Scanning {len(yaml_files)} YAML files...")
    
    for yaml_file in yaml_files:
        try:
            with open(yaml_file) as f:
                config = yaml.safe_load(f)
            
            # Check sources
            sources = config.get('sources', [])
            # Handle both list and dict formats
            if isinstance(sources, dict):
                source_ids = list(sources.keys())
            else:
                source_ids = [s.get('id') for s in sources if s.get('id')]
            
            for sid in source_ids:
                if sid in all_sources:
                    print(f"  ❌ DUPLICATE SOURCE: {sid}")
                    print(f"     File 1: {all_sources[sid]}")
                    print(f"     File 2: {yaml_file.name}")
                    return False
                all_sources[sid] = yaml_file.name
            
            # Check tools
            tools = config.get('tools', [])
            if isinstance(tools, dict):
                tool_ids = list(tools.keys())
            else:
                tool_ids = [t.get('id') for t in tools if t.get('id')]
            
            for tid in tool_ids:
                if tid in all_tools:
                    print(f"  ❌ DUPLICATE TOOL: {tid}")
                    print(f"     File 1: {all_tools[tid]}")
                    print(f"     File 2: {yaml_file.name}")
                    return False
                all_tools[tid] = yaml_file.name
        
        except Exception as e:
            print(f"  ⚠️  Error reading {yaml_file.name}: {e}")
    
    print(f"\n✅ No conflicts found!")
    print(f"   Total sources: {len(all_sources)}")
    print(f"   Total tools: {len(all_tools)}")
    print(f"\n📋 Source IDs:")
    for sid in sorted(all_sources.keys()):
        print(f"     - {sid} ({all_sources[sid]})")
    print(f"\n📋 Tool IDs:")
    for tid in sorted(all_tools.keys()):
        print(f"     - {tid} ({all_tools[tid]})")
    
    return True

# Run conflict check
conflicts_ok = check_conflicts(CONFIG_DIR)

## 6️⃣ Update Docker Compose (CRITICAL!)

Change from single file to folder path so all YAML files are loaded.

In [ ]:
print(f"📝 Docker Compose file: {DOCKER_COMPOSE_FILE}")
print(f"\n⚠️  MANUAL ACTION REQUIRED:")
print(f"   Edit: {DOCKER_COMPOSE_FILE}")
print(f"\n   Change FROM:")
print(f"     - --tools-file")
print(f"     - /config/tools.${{DNB_ENVIRONMENT:-dev}}.yaml")
print(f"\n   Change TO:")
print(f"     - --tools-file")
print(f"     - /config")
print(f"\n💡 This tells GenAI Toolbox to load ALL .yaml files from /config folder")
print(f"\n✅ After editing, continue to next cell to restart the container")

## 7️⃣ Restart GenAI Toolbox Container

Run this after updating docker-compose.dev.yml

In [ ]:
# Restart the container
toolbox_dir = WORKSPACE / "backend" / "toolbox"

print("🔄 Restarting GenAI Toolbox...")
result = subprocess.run(
    ["docker-compose", "-f", "docker-compose.dev.yml", "restart", "genai-toolbox-mcp"],
    cwd=toolbox_dir,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Container restarted successfully!")
    print("\n⏳ Waiting for container to be ready...")
    time.sleep(5)
    print("✅ Should be ready now!")
else:
    print(f"❌ Error restarting container:")
    print(result.stderr)

## 8️⃣ View Container Logs

Check that files are being loaded correctly.

In [ ]:
# Get recent logs
print("📋 Recent container logs:")
print("=" * 80)

result = subprocess.run(
    ["docker-compose", "-f", "docker-compose.dev.yml", "logs", "--tail=50", "genai-toolbox-mcp"],
    cwd=toolbox_dir,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    # Filter for relevant lines
    for line in result.stdout.split('\n'):
        if any(keyword in line.lower() for keyword in ['yaml', 'tool', 'source', 'load', 'error', 'warn']):
            print(line)
else:
    print(result.stderr)